In [1]:
from tensorflow import keras
from HelperFunctions.FileReader import FileReader
from HelperFunctions.OptimzationClass import OptmiziationClass
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# loading pre-trained tensorflow models 
lstm_d50 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_ D_v50_test.h5')
lstm_torque = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_torque_test.h5')
lstm_zone2 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 2_test.h5')
lstm_zone3 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 3_test.h5')
lstm_zone4 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 4_test.h5')
lstm_zone5 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 5_test.h5')
lstm_zone6 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 6_test.h5')
lstm_zone7 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 7_test.h5')
lstm_zone8 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_Zone 8_test.h5')
# Reading data files 
# run_name_all = ['Run1.xlsx','Run2.xlsx','Run5.xlsx','Run3.xlsx','Run4.xlsx','Run6.xlsx']
run_name_all = ['Run5.xlsx']
run_time = 29 # minutes
run_path_tsg = './TSGscreen/'
run_path_feeder = './Feederdata/'
run_path_eyecon = './Eyecondata/'
combined_datafile = pd.DataFrame()
for run_name in run_name_all:

    fr_obj = FileReader(run_name)

    tsg_data = fr_obj.read_excelFile(run_path_tsg,0)
    feeder_data = fr_obj.read_excelFile(run_path_feeder,1)
    eyecon_data = fr_obj.read_excelFile(run_path_eyecon,0)

    print(len(tsg_data))
    print(len(eyecon_data))
    print(len(feeder_data))

    combined_datafile = combined_datafile.append(fr_obj.combine_datafiles(tsg_data,eyecon_data,feeder_data))
    print(len(combined_datafile))
# Creating optimization class object 
opt_obj = OptmiziationClass(combined_datafile,(lstm_d50,lstm_torque,lstm_zone2,lstm_zone3,lstm_zone4,lstm_zone5,lstm_zone6,lstm_zone7,lstm_zone8))
# print(np.array([combined_datafile['Zone 2'].tolist()[i+1] - combined_datafile['Zone 2'].tolist()[i] for i in range(0,len(combined_datafile['Zone 2'])-1)]))
# Heat transfer of unoptimizied process
og_time, og_hl_comp, og_ta_diff, og_work_gran_arr, og_eq_hg, og_energy_used, og_work_gran_tot = opt_obj.heat_transfer(combined_datafile)
# defining optimization criteria
start_time_index = 0
opt1_time_index  = 120 #1st opt at 10 mins 5 second intervals on average i.e. 12 time steps = 1 min
opt1_time_index  = 240
# making time index array 
time_index = [0,120,240] # update this array for more/less optimizaiton steps
val = opt_obj.objective_fun1([500,1,1.75],60,120,True)
# mod_datafile, energy_used, work_granulation_total, work_granulation, equipment_heat_gained, heat_lost_compartment, time = opt_obj.objective_fun1([500,1,1.75],60,120,False)
print(val)